In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
NVIDIA GeForce RTX 4070 Ti


In [3]:
!python train.py --img 640 --batch 16 --epochs 20 --data data.yaml --cfg ./cfg/training/yolov7.yaml --weights yolov7.pt --device 0
#!python train.py --img 640 --batch 16 --epochs 10 --data ./path/to/data.yaml --cfg ./cfg/training/yolov7.yaml --weights yolov7.pt --device 0 --save_period 1


^C


In [1]:
!python test.py --data data.yaml --img 640 --batch 16 --conf 0.001 --iou 0.65 --device 0 --weights runs/train/exp2/weights/best.pt --name yolov7

Namespace(weights=['runs/train/exp2/weights/best.pt'], data='data.yaml', batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolov7', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         496        1196       0.849       0.889       0.912       0.796
      aluminium-cans         496         406       0.813       0.936       0.925       0.804
               glass         496         380       0.912        0.85       0.897       0.811
      plastic-bottle         496         410        0.82        0.88       0.915       0.773
Speed: 7.7/1.5/9.3 ms inference/NMS/total per 640x640 image at batch-siz

YOLOR  v0.1-128-ga207844 torch 2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12281.375MB)

c:\Python310\lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36503348 parameters, 6194944 gradients, 103.2 GFLOPS

val: Scanning '..\valid\labels.cache' images and labels... 496 found, 0 missing, 2 empty, 0 corrupted: 100%|██████████| 496/496 [00:00<?, ?it/s]
val: Scanning '..\valid\labels.cache' images and labels... 496 found, 0 missing, 2 empty, 0 corrupted: 100%|██████████| 496/496 [00:00<?, ?it/s]

               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95:   0%|          | 0/31 [00:00<?, ?it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95:   3%|

In [10]:
import torch
import cv2
import numpy as np
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
import os

model_path = 'runs/train/exp2/weights/best.pt'
data_yaml_path = 'data.yaml'

image_path = 'C:/Users/elvis/Downloads/prueba.png'

img_cv2 = cv2.imread(image_path)
if img_cv2 is None:
    raise FileNotFoundError(f'No se pudo encontrar la imagen en la ruta: {image_path}')

img_resized = cv2.resize(img_cv2, (640, 640))
img_resized = img_resized / 255.0
img_resized = img_resized.transpose(2, 0, 1)
img_resized = np.expand_dims(img_resized, axis=0)
img_resized = torch.tensor(img_resized, dtype=torch.float)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = attempt_load(model_path, map_location=device)
model = model.to(device)


model.eval()
with torch.no_grad():
    img_resized = img_resized.to(device)
    pred = model(img_resized)[0]
    pred = non_max_suppression(pred, 0.25, 0.45)


for det in pred:
    if len(det):
        det[:, :4] = scale_coords(img_resized.shape[2:], det[:, :4], img_cv2.shape).round()
        for *xyxy, conf, cls in det:
            label = f'{model.names[int(cls)]} {conf:.2f}'
            xyxy = [int(x) for x in xyxy]
            cv2.rectangle(img_cv2, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
            cv2.putText(img_cv2, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


output_folder = 'runs/img_prueba'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
output_image_path = os.path.join(output_folder, 'detected_image_3.jpg')
cv2.imwrite(output_image_path, img_cv2)
cv2.imshow('Detections', img_cv2)
cv2.waitKey(0)
cv2.destroyAllWindows()


Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [7]:
import torch
import cv2
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
import os

model_path = 'runs/train/exp2/weights/best.pt'
data_yaml_path = 'data.yaml'

# URL de la imagen de internet
image_url = 'https://pic.52112.com/180713/JPG-180713_870/whwK6ujFSZ_small.jpg'

response = requests.get(image_url)
img = Image.open(BytesIO(response.content)).convert('RGB')
img_cv2 = np.array(img)
img_cv2 = img_cv2[:, :, ::-1].copy()  # Convertir de RGB a BGR

img_resized = cv2.resize(img_cv2, (640, 640))
img_resized = img_resized / 255.0
img_resized = img_resized.transpose(2, 0, 1)
img_resized = np.expand_dims(img_resized, axis=0)
img_resized = torch.tensor(img_resized, dtype=torch.float)

# Cargar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = attempt_load(model_path, map_location=device)
model = model.to(device)

#
# Realizar la detección
model.eval()
with torch.no_grad():
    img_resized = img_resized.to(device)
    pred = model(img_resized)[0]
    pred = non_max_suppression(pred, 0.25, 0.45)

for det in pred:
    if len(det):
        det[:, :4] = scale_coords(img_resized.shape[2:], det[:, :4], img_cv2.shape).round()
        for *xyxy, conf, cls in det:
            label = f'{model.names[int(cls)]} {conf:.2f}'
            xyxy = [int(x) for x in xyxy]
            cv2.rectangle(img_cv2, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
            cv2.putText(img_cv2, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

output_folder = 'runs/img_prueba'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
output_image_path = os.path.join(output_folder, 'detected_image_2.jpg')
cv2.imwrite(output_image_path, img_cv2)
cv2.imshow('Detections', img_cv2)
cv2.waitKey(0)
cv2.destroyAllWindows()


Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [1]:
import torch
import cv2
import numpy as np
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
import os

model_path = 'runs/train/exp2/weights/best.pt'
video_path = 'C:/modelo/yolov7/runs/videos_pruebas/prueba_3.mp4'
output_path = 'C:/modelo/yolov7/runs/videos_pruebas/procesado_prueba_3.mp4'

# Cargar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = attempt_load(model_path, map_location=device)
model = model.to(device)

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_resized = cv2.resize(frame, (640, 640))
    img_resized = img_resized / 255.0
    img_resized = img_resized.transpose(2, 0, 1)
    img_resized = np.expand_dims(img_resized, axis=0)
    img_resized = torch.tensor(img_resized, dtype=torch.float).to(device)
    model.eval()
    with torch.no_grad():
        pred = model(img_resized)[0]
        pred = non_max_suppression(pred, 0.25, 0.45)

    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img_resized.shape[2:], det[:, :4], frame.shape).round()
            for *xyxy, conf, cls in det:
                label = f'{model.names[int(cls)]} {conf:.2f}'
                xyxy = [int(x) for x in xyxy]
                cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
                cv2.putText(frame, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
#cerrar
cap.release()
out.release()
cv2.destroyAllWindows()


Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


c:\Python310\lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
